In [1]:
%load_ext autoreload
%autoreload 2

# 1. Create dataset
Use Sam to extract proposals from input images (100 images) then for proposals with same regions as the ground truth masks, they are considered as positive sameple to the templates ( use the BlenderProc 42 templates), negative otherwise

Run through 100 images - then return positive proposals and negative proposals

Use IoU >0.5 to get the postive proposals

In [28]:
from src.model.constrastive_learning import extract_dataset

dataset="icbin"
data_type="test"
scene_id=1
pos_proposals, neg_proposals = extract_dataset(dataset, data_type, scene_id) # Take 2.21 minutes

[2024-08-13 12:49:14,582] [INFO] src.model.constrastive_learning: loading sam
[2024-08-13 12:49:14,583] [INFO] root: Loading SAM model from datasets/bop23_challenge/pretrained/segment-anything
[2024-08-13 12:49:17,149] [INFO] root: Init CustomSamAutomaticMaskGenerator done!
[2024-08-13 12:49:21,764] [INFO] src.model.constrastive_learning: The best for 0th mask is at index 3 with an IoU of 0.884139219808456
[2024-08-13 12:49:21,795] [INFO] src.model.constrastive_learning: The best for 1th mask is at index 29 with an IoU of 0.8490939977349944
[2024-08-13 12:49:21,827] [INFO] src.model.constrastive_learning: The best for 2th mask is at index 30 with an IoU of 0.7815985130111525
[2024-08-13 12:49:21,858] [INFO] src.model.constrastive_learning: The best for 3th mask is at index 55 with an IoU of 0.8901230861603122
[2024-08-13 12:49:21,890] [INFO] src.model.constrastive_learning: The best for 4th mask is at index 23 with an IoU of 0.6603650586701434
[2024-08-13 12:49:21,922] [INFO] src.model

In [27]:
len(pos_proposals[0])

19

In [16]:
all_pos_proposals = [item for sublist in pos_proposals for item in sublist]
all_neg_proposals = [item for sublist in neg_proposals for item in sublist]

In [22]:
import os 

obj_id = 1
template_path = f"datasets/bop23_challenge/datasets/templates_pyrender/{dataset}/obj_{obj_id:06d}"
template_files = sorted(glob.glob(os.path.join(template_path, "*.png")), key=os.path.getmtime)
templates = [np.array(Image.open(template_file).convert("RGB"))[ :, :, :3] for template_file in template_files] # This image has 4 channels- the last one is not crucial - maybe about opacity

# 2. Process dataset
Build train_dataset as postive, negative pairs
Then build train_loader

Organize your images into classes. You'll need positive pairs (same class) and negative pairs (different classes) for training.
Resize/pad image to 224*224  then /255.0
then transform the image (also with std, mean as in ImageNet)

In [ ]:
from src.model.constrastive_learning import PairedDataset

# Custom dataset for paired images

# 3. Model

Two Approaches

    Extract features from Dinov2 as in cnos
    
    Extract BoW vectors as in FoundPose


In [ ]:
from src.model.constrastive_learning import ContrastiveModel

model = ContrastiveModel()


# 4. Implement contrastive loss function


In [23]:
# Contrastive loss function
class ContrastiveLoss(nn.Module):
    def __init__(self, margin=1.0):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin

    def forward(self, output1, output2, label):
        euclidean_distance = nn.functional.pairwise_distance(output1, output2)
        loss = torch.mean((1-label) * torch.pow(euclidean_distance, 2) +
                          (label) * torch.pow(torch.clamp(self.margin - euclidean_distance, min=0.0), 2))
        return loss

NameError: name 'nn' is not defined

# 5. Create data pairs for training


# 6. Train the model